In [2]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hail-m.us-central1-f.c.cncd-cncd.internal:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /hail-20210211-1104-0.2.61-3c86d3ba497a.log


In [ ]:
files = ["gs://pgr-wes-r20/GL_by_chrom/CNCD_Freeze_Two.6.GL.vcf.gz", "gs://pgr_wes_30k/GRCh38/chr6_30k_grch38.vcf.bgz",
        "gs://pgr-wgs-8k/VCF/vcf_DP10/pgr7k_chr6.vcf.gz", "gs://pgr-wgs-2k/chr6_PAK_indv.recode.vcf.gz"]

recode = {f"{i}":f"chr{i}" for i in (list(range(1, 23)) + ['X', 'Y'])}

pos = 'chr6:3105564-3105565'

for file in files:
    data = hl.import_vcf(file, reference_genome="GRCh38", force_bgz=True, contig_recoding=recode)
    ds_result1 = hl.filter_intervals(data, [hl.parse_locus_interval(pos, reference_genome='GRCh38')])
    ds_result1.rows().show()
    if ds_result1.count_rows() != 0:
        print('Annotate Heterozygous Alleles')
        ds_result1 = ds_result1.annotate_rows(hets = hl.agg.filter(ds_result1.GT.is_het(),hl.agg.collect(ds_result1.s)))
        ds_result1.count()
        print('Annotate N Hets')
        ds_result4 = ds_result1.annotate_rows(n_het = hl.agg.count_where(ds_result1.GT.is_het()))
        ds_result4.count()
        print('Annotate Homozygous Alleles')
        ds_result3 = ds_result4.annotate_rows(homs = hl.agg.filter(ds_result4.GT.is_hom_var(),hl.agg.collect(ds_result4.s)))
        ds_result3.count()
        print('Annotating N Homs')
        ds_result5 = ds_result3.annotate_rows(n_hom = hl.agg.count_where(ds_result3.GT.is_hom_var()))

        ds_result5.rows().export("gs://test1haseeb/Kashif-Lookup/"+file+pos+".tsv")
    

2021-02-11 11:06:35 Hail: INFO: Coerced sorted dataset
2021-02-11 11:06:35 Hail: INFO: reading 1 of 79 data partitions


+---------------+-------------+-------------------+----------+----------+
| locus         | alleles     | rsid              |     qual | filters  |
+---------------+-------------+-------------------+----------+----------+
| locus<GRCh38> | array<str>  | str               |  float64 | set<str> |
+---------------+-------------+-------------------+----------+----------+
| chr6:3105564  | ["GGA","G"] | "6_3105564_GGA_G" | 1.53e+03 | NA       |
+---------------+-------------+-------------------+----------+----------+

+----------------+--------------+
| info.AF        | info.AQ      |
+----------------+--------------+
| array<float64> | array<int32> |
+----------------+--------------+
| [2.60e-05]     | [1525]       |
+----------------+--------------+

2021-02-11 11:07:05 Hail: INFO: Coerced sorted dataset
2021-02-11 11:07:05 Hail: INFO: reading 1 of 79 data partitions


Annotate Heterozygous Alleles


2021-02-11 11:07:25 Hail: INFO: Coerced sorted dataset
2021-02-11 11:07:26 Hail: INFO: reading 1 of 79 data partitions


...

2021-02-11 11:08:09 Hail: INFO: Coerced sorted dataset
2021-02-11 11:08:09 Hail: INFO: reading 1 of 79 data partitions


Annotating N Homs


2021-02-11 11:08:30 Hail: INFO: Coerced sorted dataset
2021-02-11 11:08:30 Hail: INFO: reading 1 of 79 data partitions
2021-02-11 11:08:57 Hail: INFO: merging 1 files totalling 170...
2021-02-11 11:08:58 Hail: INFO: while writing:
    gs://test1haseeb/Kashif-Lookup/gs://pgr-wes-r20/GL_by_chrom/CNCD_Freeze_Two.6.GL.vcf.gzchr6:3105564-3105565.tsv
  merge time: 425.393ms
2021-02-11 11:09:18 Hail: INFO: Coerced sorted dataset
2021-02-11 11:09:18 Hail: INFO: reading 1 of 140 data partitions


+---------------+------------+------+---------+----------+---------+----------+
| locus         | alleles    | rsid |    qual | filters  | info.AC | info.aaf |
+---------------+------------+------+---------+----------+---------+----------+
| locus<GRCh38> | array<str> | str  | float64 | set<str> |   int32 |  float64 |
+---------------+------------+------+---------+----------+---------+----------+
+---------------+------------+------+---------+----------+---------+----------+

+---------------+------------+---------------+------------+------------+
| info.callRate | info.hetRA | info.hg19_uid | info.homAA | info.homRR |
+---------------+------------+---------------+------------+------------+
|       float64 |      int32 | str           |      int32 |      int32 |
+---------------+------------+---------------+------------+------------+
+---------------+------------+---------------+------------+------------+

+---------------+---------------+---------------+---------------+
| info.maxDepth | info.maxHetAB | info.maxHomAB | info.medDepth |
+---------------+---------------+---------------+---------------+
|         int32 |       float64 |       float64 |         int32 |
+---------------+---------------+---------------+---------------+
+---------------+---------------+---------------+---------------+

+---------------+---------------+---------------+---------------+
| info.medHetAB | info.medHomAB | info.minDpeth | info.minHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+---------------+
+---------------+---------------+---------------+---------------+

+---------------+-------------+----------------------+--------------+
| info.minHomAB | info.noCall | info.oldMultiAllelic | info.pvalHwe |
+---------------+-------------+----------------------+--------------+
|       float64 |       int32 | str                  |      float64 |
+---------------+-------------+----------------------+--------------+
+---------------+-------------+----------------------+--------------+

2021-02-11 11:09:41 Hail: INFO: Coerced sorted dataset
2021-02-11 11:09:41 Hail: INFO: reading 1 of 140 data partitions
2021-02-11 11:11:04 Hail: INFO: Coerced sorted dataset
2021-02-11 11:11:04 Hail: INFO: reading 1 of 199 data partitions


In [10]:
ds_result1.count()

2021-02-11 09:29:19 Hail: INFO: Coerced sorted dataset
2021-02-11 09:29:19 Hail: INFO: reading 1 of 202 data partitions


(2, 30833)

In [13]:
# if ds_result1.count()[0] != 0:
ds_result1.count_rows()

2021-02-11 09:33:31 Hail: INFO: Coerced sorted dataset
2021-02-11 09:33:31 Hail: INFO: reading 1 of 202 data partitions


2


In [2]:
data6 = hl.read_matrix_table("gs://test1haseeb/GOLGA6L6_20K_vep.mt")
data6.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AF: array<float64>, 
        AQ: array<int32>, 
        hets: array<str>, 
        n_het: int32, 
        homs: array<str>, 
        n_hom: int32
    }
    'vep': struct {
        assembly_name: str, 
        allele_string: str, 
        ancestral: str, 
        colocated_variants: array<struct {
            aa_allele: str, 
            aa_maf: float64, 
            afr_allele: str, 
            afr_maf: float64, 
            allele_string: str, 
            amr_allele: str, 
            amr_maf: float64, 
            clin_sig: array<str>, 
            end: int32, 
            eas_allele: str, 
            eas_maf: float64, 
            ea_allele: str, 
         

In [17]:
data6 = hl.read_matrix_table("gs://test1haseeb/GOLGA6L6_20K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('GOLGA6L6_20K_vep.xlsx')
!gsutil cp GOLGA6L6_20K_vep.xlsx gs://test1haseeb/var-lookup/

+----------------+------------+-----------------------------------+----------+
| locus          | alleles    | rsid                              |     qual |
+----------------+------------+-----------------------------------+----------+
| locus<GRCh38>  | array<str> | str                               |  float64 |
+----------------+------------+-----------------------------------+----------+
| chr15:20534182 | ["C","A"]  | "15_20534182_C_A"                 | 5.90e+01 |
| chr15:20534186 | ["C","T"]  | "15_20534186_C_T"                 | 6.80e+01 |
| chr15:20534194 | ["C","T"]  | "15_20534194_C_T"                 | 8.10e+01 |
| chr15:20534198 | ["C","A"]  | "15_20534198_C_G;15_20534198_C_A" | 9.60e+01 |
| chr15:20534198 | ["C","G"]  | "15_20534198_C_G;15_20534198_C_A" | 9.60e+01 |
| chr15:20534199 | ["C","A"]  | "15_20534199_C_A"                 | 5.90e+01 |
| chr15:20534207 | ["C","T"]  | "15_20534207_C_T"                 | 5.30e+01 |
| chr15:20534209 | ["G","A"]  | "15_20534209_G_A"                 | 6.70e+01 |
| chr15:20534219 | ["C","T"]  | "15_20534219_C_T"                 | 7.80e+01 |
| chr15:20534226 | ["C","T"]  | "15_20534226_C_T"                 | 6.10e+01 |
+----------------+------------+-----------------------------------+----------+

+----------+---------------------+--------------+------------+------------+
| filters  | info.AF             | info.AQ      | info.hets  | info.n_het |
+----------+---------------------+--------------+------------+------------+
| set<str> | array<float64>      | array<int32> | array<str> |      int32 |
+----------+---------------------+--------------+------------+------------+
| NA       | [2.60e-05]          | [59]         | NA         |          0 |
| NA       | [5.10e-05]          | [68]         | NA         |          0 |
| NA       | [2.60e-05]          | [81]         | NA         |          0 |
| NA       | [5.10e-05,2.60e-05] | [96,39]      | NA         |          0 |
| NA       | [5.10e-05,2.60e-05] | [96,39]      | NA         |          0 |
| NA       | [2.60e-05]          | [59]         | NA         |          0 |
| NA       | [5.10e-05]          | [53]         | NA         |          0 |
| NA       | [5.10e-05]          | [67]         | NA         |          0 |
| NA       | [2.60e-05]          | [78]         | NA         |          0 |
| NA       | [2.60e-05]          | [61]         | NA         |          0 |
+----------+---------------------+--------------+------------+------------+

+------------+------------+---------------+---------------------+
| info.homs  | info.n_hom | info.GeneName | info.EnsembleGeneId |
+------------+------------+---------------+---------------------+
| array<str> |      int32 | array<str>    | array<str>          |
+------------+------------+---------------+---------------------+
| NA         |          0 | ["GOLGA6L6"]  | ["ENSG00000277322"] |
| NA         |          0 | ["GOLGA6L6"]  | ["ENSG00000277322"] |
| NA         |          0 | ["GOLGA6L6"]  | ["ENSG00000277322"] |
| NA         |          0 | ["GOLGA6L6"]  | ["ENSG00000277322"] |
| NA         |          0 | ["GOLGA6L6"]  | ["ENSG00000277322"] |
| NA         |          0 | ["GOLGA6L6"]  | ["ENSG00000277322"] |
| NA         |          0 | ["GOLGA6L6"]  | ["ENSG00000277322"] |
| NA         |          0 | ["GOLGA6L6"]  | ["ENSG00000277322"] |
| NA         |          0 | ["GOLGA6L6"]  | ["ENSG00000277322"] |
| NA         |          0 | ["GOLGA6L6"]  | ["ENSG00000277322"] |
+------------+------------+---------------+---------------------+

+-------------------------+------------------------+--------------------------+
| info.variantConsequence | info.consequence_terms | info.transcript_affected |
+-------------------------+------------------------+--------------------------+
| str                     | array<array<str>>      | array<str>               |
+-------------------------+------------------------+--------------------------+
| "intron_variant"        | [["intron_variant"]]   | ["ENST00

Copying file://GOLGA6L6_20K_vep.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][ 88.5 KiB/ 88.5 KiB]                                                
Operation completed over 1 objects/88.5 KiB.                                     


In [18]:
data6 = hl.read_matrix_table("gs://test1haseeb/GALNT9_30K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('GALNT9_30K_vep.xlsx')
!gsutil cp GALNT9_30K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/GALNT9_30K_vep.tsv")

+-----------------+------------+------+----------+----------+---------+
| locus           | alleles    | rsid |     qual | filters  | info.AC |
+-----------------+------------+------+----------+----------+---------+
| locus<GRCh38>   | array<str> | str  |  float64 | set<str> |   int32 |
+-----------------+------------+------+----------+----------+---------+
| chr12:132197059 | ["C","T"]  | NA   | 2.14e+03 | {}       |       2 |
| chr12:132197060 | ["G","A"]  | NA   | 4.42e+03 | {}       |       1 |
| chr12:132197060 | ["G","C"]  | NA   | 4.42e+03 | {}       |       0 |
| chr12:132197064 | ["C","T"]  | NA   | 8.46e+02 | {}       |       1 |
| chr12:132197067 | ["C","T"]  | NA   | 2.86e+03 | {}       |       2 |
| chr12:132197075 | ["C","T"]  | NA   | 3.74e+03 | {}       |       1 |
| chr12:132197076 | ["G","A"]  | NA   | 9.25e+03 | {}       |       1 |
| chr12:132197088 | ["G","T"]  | NA   | 3.67e+02 | {}       |       1 |
| chr12:132197092 | ["C","T"]  | NA   | 5.43e+05 | {}       |       6 |
| chr12:132197093 | ["G","A"]  | NA   | 9.38e+03 | {}       |       1 |
+-----------------+------------+------+----------+----------+---------+

+----------+---------------+------------+--------------------+------------+
| info.aaf | info.callRate | info.hetRA | info.hg19_uid      | info.homAA |
+----------+---------------+------------+--------------------+------------+
|  float64 |       float64 |      int32 | str                |      int32 |
+----------+---------------+------------+--------------------+------------+
| 3.24e-05 |      1.00e+00 |          2 | "12:132681604:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681605:G:A" |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "12:132681605:G:C" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681609:C:T" |          0 |
| 3.24e-05 |      1.00e+00 |          2 | "12:132681612:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681620:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681621:G:A" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681633:G:T" |          0 |
| 9.73e-05 |      1.00e+00 |          6 | "12:132681637:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681638:G:A" |          0 |
+----------+---------------+------------+--------------------+------------+

+------------+---------------+---------------+---------------+---------------+
| info.homRR | info.maxDepth | info.maxHetAB | info.maxHomAB | info.medDepth |
+------------+---------------+---------------+---------------+---------------+
|      int32 |         int32 |       float64 |       float64 |         int32 |
+------------+---------------+---------------+---------------+---------------+
|      30825 |           223 |            NA |            NA |            39 |
|      30832 |           223 |            NA |            NA |            39 |
|      30833 |           223 |            NA |            NA |            39 |
|      30830 |           223 |            NA |            NA |            40 |
|      30829 |           223 |            NA |            NA |            40 |
|      30831 |           223 |            NA |            NA |            40 |
|      30831 |           223 |            NA |            NA |            40 |
|      30832 |           223 |            NA |            NA |            40 |
|      30827 |           223 |      5.24e-01 |            NA |            40 |
|      30832 |           223 |            NA |            NA |            40 |
+------------+---------------+---------------+---------------+---------------+

+---------------+---------------+---------------+---------------+
| info.medHetAB | info.medHomAB | info.minDpeth | info.minHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+---------------+
|      4.30e-01 |            NA |  

Copying file://GALNT9_30K_vep.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][289.7 KiB/289.7 KiB]                                                
Operation completed over 1 objects/289.7 KiB.                                    


In [19]:
data6 = hl.read_matrix_table("gs://test1haseeb/KIR3DL1_20K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('KIR3DL1_20K_vep.xlsx')
!gsutil cp KIR3DL1_20K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/KIR3DL1_20K_vep.tsv")

+----------------+------------+------+----------+
| locus          | alleles    | rsid |     qual |
+----------------+------------+------+----------+
| locus<GRCh38>  | array<str> | str  |  float64 |
+----------------+------------+------+----------+
| chr19:54816469 | ["G","A"]  | NA   | 3.27e+02 |
| chr19:54816471 | ["G","A"]  | NA   | 1.68e+03 |
| chr19:54816471 | ["G","T"]  | NA   | 1.68e+03 |
| chr19:54816472 | ["C","T"]  | NA   | 1.13e+03 |
| chr19:54816474 | ["C","T"]  | NA   | 5.60e+02 |
| chr19:54816475 | ["A","G"]  | NA   | 6.99e+05 |
| chr19:54816478 | ["C","T"]  | NA   | 1.97e+02 |
| chr19:54816479 | ["G","A"]  | NA   | 4.90e+03 |
| chr19:54816484 | ["T","A"]  | NA   | 6.59e+01 |
| chr19:54816491 | ["C","A"]  | NA   | 9.28e+03 |
+----------------+------------+------+----------+

+---------------------------------+---------+----------+---------------+
| filters                         | info.AC | info.aaf | info.callRate |
+---------------------------------+---------+----------+---------------+
| set<str>                        |   int32 |  float64 |       float64 |
+---------------------------------+---------+----------+---------------+
| {"VQSRTrancheSNP99.60to99.80"}  |       2 | 4.14e-05 |      7.83e-01 |
| {}                              |       5 | 8.11e-05 |      1.00e+00 |
| {}                              |       0 | 0.00e+00 |      1.00e+00 |
| {}                              |       1 | 2.07e-05 |      7.82e-01 |
| {"VQSRTrancheSNP99.90to99.95"}  |      11 | 2.28e-04 |      7.82e-01 |
| {"VQSRTrancheSNP99.95to100.00"} |    7733 | 1.69e-01 |      7.42e-01 |
| {"VQSRTrancheSNP99.80to99.90"}  |       2 | 4.14e-05 |      7.84e-01 |
| {}                              |       2 | 4.14e-05 |      7.84e-01 |
| {"VQSRTrancheSNP99.60to99.80"}  |       2 | 4.13e-05 |      7.84e-01 |
| {}                              |       6 | 9.73e-05 |      1.00e+00 |
+---------------------------------+---------+----------+---------------+

+------------+-------------------+------------+------------+---------------+
| info.hetRA | info.hg19_uid     | info.homAA | info.homRR | info.maxDepth |
+------------+-------------------+------------+------------+---------------+
|      int32 | str               |      int32 |      int32 |         int32 |
+------------+-------------------+------------+------------+---------------+
|          0 | "19:55327924:G:A" |          1 |      24133 |           506 |
|          1 | "19:55327926:G:A" |          2 |      30830 |           506 |
|          0 | "19:55327926:G:T" |          0 |      30831 |           506 |
|          1 | "19:55327927:C:T" |          0 |      24121 |           506 |
|          3 | "19:55327929:C:T" |          4 |      24096 |           506 |
|        223 | "19:55327930:A:G" |       3755 |      18894 |           506 |
|          2 | "19:55327933:C:T" |          0 |      24161 |           506 |
|          0 | "19:55327934:G:A" |          1 |      24180 |           506 |
|          0 | "19:55327939:T:A" |          1 |      24185 |           506 |
|          2 | "19:55327946:C:A" |          2 |      30825 |           506 |
+------------+-------------------+------------+------------+---------------+

+---------------+---------------+---------------+---------------+
| info.maxHetAB | info.maxHomAB | info.medDepth | info.medHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+---------------+
|            NA |            NA |            36 |            NA |
|            NA |            NA |            22 |            NA |
|            NA |            NA |            22 |            NA |
|            NA |            NA |            36 |            NA |
|      6.00e-01 |      1.00e+00 |            36 |      5.00e-01 |
|      8.22e-02 |      8.00e-01 |            38 |      8.22e-02 |
|            NA |            NA |            36 |      6.00e-01 |
|    

Copying file://KIR3DL1_20K_vep.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][125.9 KiB/125.9 KiB]                                                
Operation completed over 1 objects/125.9 KiB.                                    


In [20]:
data6 = hl.read_matrix_table("gs://test1haseeb/LYPLAL1_20K.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('LYPLAL1_20K_vep.xlsx')
!gsutil cp LYPLAL1_20K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/LYPLAL1_20K_vep.tsv")

+----------------+-------------+------+----------+----------+---------+
| locus          | alleles     | rsid |     qual | filters  | info.AC |
+----------------+-------------+------+----------+----------+---------+
| locus<GRCh38>  | array<str>  | str  |  float64 | set<str> |   int32 |
+----------------+-------------+------+----------+----------+---------+
| chr1:219173875 | ["G","C"]   | NA   | 1.45e+03 | {}       |       1 |
| chr1:219173878 | ["A","C"]   | NA   | 1.24e+04 | {}       |       1 |
| chr1:219173879 | ["G","A"]   | NA   | 1.32e+04 | {}       |       0 |
| chr1:219173879 | ["G","C"]   | NA   | 1.32e+04 | {}       |       1 |
| chr1:219173885 | ["T","A"]   | NA   | 3.47e+03 | {}       |       3 |
| chr1:219173892 | ["T","C"]   | NA   | 7.37e+03 | {}       |       7 |
| chr1:219173904 | ["C","T"]   | NA   | 3.34e+03 | {}       |       1 |
| chr1:219173904 | ["CGG","C"] | NA   | 3.34e+03 | {}       |       0 |
| chr1:219173905 | ["G","A"]   | NA   | 3.98e+03 | {}       |       1 |
| chr1:219173905 | ["G","C"]   | NA   | 3.98e+03 | {}       |       0 |
+----------------+-------------+------+----------+----------+---------+

+----------+---------------+------------+-----------------------+------------+
| info.aaf | info.callRate | info.hetRA | info.hg19_uid         | info.homAA |
+----------+---------------+------------+-----------------------+------------+
|  float64 |       float64 |      int32 | str                   |      int32 |
+----------+---------------+------------+-----------------------+------------+
| 1.62e-05 |      1.00e+00 |          1 | "1:219347217:G:C"     |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347220:A:C"     |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "1:219347221:G:A"     |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347221:G:C"     |          0 |
| 4.86e-05 |      1.00e+00 |          1 | "1:219347227:T:A"     |          1 |
| 1.14e-04 |      1.00e+00 |          7 | "1:219347234:T:C"     |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347246:CGG:TGG" |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "1:219347246:CGG:C"   |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347247:G:A"     |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "1:219347247:G:C"     |          0 |
+----------+---------------+------------+-----------------------+------------+

+------------+---------------+---------------+---------------+---------------+
| info.homRR | info.maxDepth | info.maxHetAB | info.maxHomAB | info.medDepth |
+------------+---------------+---------------+---------------+---------------+
|      int32 |         int32 |       float64 |       float64 |         int32 |
+------------+---------------+---------------+---------------+---------------+
|      30831 |           443 |            NA |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30833 |           443 |            NA |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30831 |           443 |            NA |            NA |            48 |
|      30826 |           443 |      4.50e-01 |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30833 |           443 |            NA |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30833 |           443 |            NA |            NA |            48 |
+------------+---------------+---------------+---------------+---------------+

+---------------+---------------+---------------+---------------+
| info.medHetAB | info.medHomAB | info.minDpeth | info.minHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+----

Copying file://LYPLAL1_20K_vep.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][104.1 KiB/104.1 KiB]                                                
Operation completed over 1 objects/104.1 KiB.                                    


In [21]:
data6 = hl.read_matrix_table("gs://test1haseeb/LYPLAL1_30K_vep.mt")
data1 = data6.annotate_rows(
    info=data6.info.annotate(GeneName=data6.vep.transcript_consequences.gene_symbol,
                             EnsembleGeneId=data6.vep.transcript_consequences.gene_id,
                             variantConsequence=data6.vep.most_severe_consequence,
                             consequence_terms=data6.vep.transcript_consequences.consequence_terms,
                             transcript_affected=data6.vep.transcript_consequences.transcript_id,
                             lof=data6.vep.transcript_consequences.lof,
                             lof_filter=data6.vep.transcript_consequences.lof_filter,
                             biotype=data6.vep.transcript_consequences.biotype,
                             canonical=data6.vep.transcript_consequences.canonical
                             # should we add csq_score????
                               ))
data1.rows().show()
data2 = data1.rows().to_pandas()
data2.to_excel('LYPLAL1_30K_vep.xlsx')
!gsutil cp LYPLAL1_30K_vep.xlsx gs://test1haseeb/var-lookup/
# data1.rows().export("gs://test1haseeb/var-lookup/LYPLAL1_30K_vep.tsv")

+-----------------+------------+------+----------+----------+---------+
| locus           | alleles    | rsid |     qual | filters  | info.AC |
+-----------------+------------+------+----------+----------+---------+
| locus<GRCh38>   | array<str> | str  |  float64 | set<str> |   int32 |
+-----------------+------------+------+----------+----------+---------+
| chr12:132197059 | ["C","T"]  | NA   | 2.14e+03 | {}       |       2 |
| chr12:132197060 | ["G","A"]  | NA   | 4.42e+03 | {}       |       1 |
| chr12:132197060 | ["G","C"]  | NA   | 4.42e+03 | {}       |       0 |
| chr12:132197064 | ["C","T"]  | NA   | 8.46e+02 | {}       |       1 |
| chr12:132197067 | ["C","T"]  | NA   | 2.86e+03 | {}       |       2 |
| chr12:132197075 | ["C","T"]  | NA   | 3.74e+03 | {}       |       1 |
| chr12:132197076 | ["G","A"]  | NA   | 9.25e+03 | {}       |       1 |
| chr12:132197088 | ["G","T"]  | NA   | 3.67e+02 | {}       |       1 |
| chr12:132197092 | ["C","T"]  | NA   | 5.43e+05 | {}       |       6 |
| chr12:132197093 | ["G","A"]  | NA   | 9.38e+03 | {}       |       1 |
+-----------------+------------+------+----------+----------+---------+

+----------+---------------+------------+--------------------+------------+
| info.aaf | info.callRate | info.hetRA | info.hg19_uid      | info.homAA |
+----------+---------------+------------+--------------------+------------+
|  float64 |       float64 |      int32 | str                |      int32 |
+----------+---------------+------------+--------------------+------------+
| 3.24e-05 |      1.00e+00 |          2 | "12:132681604:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681605:G:A" |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "12:132681605:G:C" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681609:C:T" |          0 |
| 3.24e-05 |      1.00e+00 |          2 | "12:132681612:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681620:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681621:G:A" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681633:G:T" |          0 |
| 9.73e-05 |      1.00e+00 |          6 | "12:132681637:C:T" |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "12:132681638:G:A" |          0 |
+----------+---------------+------------+--------------------+------------+

+------------+---------------+---------------+---------------+---------------+
| info.homRR | info.maxDepth | info.maxHetAB | info.maxHomAB | info.medDepth |
+------------+---------------+---------------+---------------+---------------+
|      int32 |         int32 |       float64 |       float64 |         int32 |
+------------+---------------+---------------+---------------+---------------+
|      30825 |           223 |            NA |            NA |            39 |
|      30832 |           223 |            NA |            NA |            39 |
|      30833 |           223 |            NA |            NA |            39 |
|      30830 |           223 |            NA |            NA |            40 |
|      30829 |           223 |            NA |            NA |            40 |
|      30831 |           223 |            NA |            NA |            40 |
|      30831 |           223 |            NA |            NA |            40 |
|      30832 |           223 |            NA |            NA |            40 |
|      30827 |           223 |      5.24e-01 |            NA |            40 |
|      30832 |           223 |            NA |            NA |            40 |
+------------+---------------+---------------+---------------+---------------+

+---------------+---------------+---------------+---------------+
| info.medHetAB | info.medHomAB | info.minDpeth | info.minHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+---------------+
|      4.30e-01 |            NA |  

Copying file://LYPLAL1_30K_vep.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][289.7 KiB/289.7 KiB]                                                
Operation completed over 1 objects/289.7 KiB.                                    
